In [1]:
import pandas as pd
import re

# Tiền xử lý trước khi schema matching

In [16]:
# Movie Info
df_movielen_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/movies.csv")
df_movielen_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Ta thấy title có cấu trúc title + (năm ra mắt phim) => tách thành 2 trường title và year

In [17]:
# Split year
import re
df_movielen_movie["year"]=[re.findall(r'(\(\d{4}\))', i) for i in df_movielen_movie["title"]]

In [18]:
count=0
print("Những trường hợp ngoại lệ:")
for idx, i in enumerate(df_movielen_movie["year"]):
    if len(i)==1:
        count+=1
    if len(i)>1:
        print("Title: "+df_movielen_movie.iloc[idx]["title"])
print("Số bản ghi có chứa năm ra mắt: "+str(count))

Những trường hợp ngoại lệ:
Title: The Devotion of Suspect X (2017) (2017)
Title: Don Quixote (1973) (1973)
Số bản ghi có chứa năm ra mắt: 62011


Ta thấy những trường hợp ngoại lệ vẫn có thể suy ra được năm phim ra mắt

In [19]:
def year_preprocess(year_list):
    if len(year_list)>=1:
        # Loại bỏ dấu ngoặc đơn
        return int(year_list[0][1:-1])
    else:
        return None
df_movielen_movie["year"]=[year_preprocess(i) for i in df_movielen_movie["year"]]
# Loại bỏ năm trong title
df_movielen_movie["title"]=[re.sub(r'(\(\d{4}\))', '', i) for i in df_movielen_movie["title"]]

Ta tổng hợp lại rating bằng cách tính avgrating và ratecount trong bảng df_movielen_rating 

In [60]:
df_movielen_rating = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/ratings.csv")
df_movielen_statistic = pd.DataFrame()
df_movielen_statistic["movieId"] = df_movielen_rating.groupby("movieId")["rating"].mean().keys()
df_movielen_statistic["avgrating"] = df_movielen_rating.groupby("movieId")["rating"].mean().values
df_movielen_statistic["ratecount"] = df_movielen_rating.groupby("movieId")["rating"].count().values
df_movielen_statistic.head()

,movieId,avgrating,ratecount
0,1,3.893708,57309
1,2,3.251527,24228
2,3,3.142028,11804
3,4,2.853547,2523
4,5,3.058434,11714


In [62]:
df_movielen_final = pd.concat([df_movielen_movie, df_movielen_statistic], axis=1)

In [65]:
df_movielen_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/movies_preprocess_1.csv", index=False)